In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import sys
import os

import os.path as osp

import time
import yaml
import torch

import smplx

from utils import JointMapper
from cmd_parser import parse_config
from data_parser import create_dataset
from fit_single_frame import fit_single_frame

from camera import create_camera
from prior import create_prior

torch.backends.cudnn.enabled = False

In [6]:
!pwd

/home/frg08/Code/smplify-x/smplifyx


In [5]:
args = {'data_folder': 'data', 'max_persons': 3, 'config': 'cfg_files/fit_smpl.yaml', 'loss_type': 'smplify', 
        'interactive': True, 'save_meshes': True, 'visualize': True, 'degrees': [0, 90, 180, 270], 
        'use_cuda': True, 'dataset': 'openpose', 'joints_to_ign': [1, 9, 12], 'output_folder': 'output', 
        'img_folder': 'images', 'keyp_folder': 'keypoints', 'summary_folder': 'summaries', 
        'result_folder': 'results', 'mesh_folder': 'meshes', 'gender_lbl_type': 'none', 'gender': 'neutral', 
        'float_dtype': 'float32', 'model_type': 'smpl', 'camera_type': 'persp', 'optim_jaw': True, 
        'optim_hands': True, 'optim_expression': True, 'optim_shape': True, 
        'model_folder': '/home/frg08/Code/smplify-x/smpl/models/smpl_female.pkl', 'use_joints_conf': True, 
        'batch_size': 1, 'num_gaussians': 8, 'use_pca': True, 'num_pca_comps': 12, 'flat_hand_mean': False, 
        'body_prior_type': 'l2', 'left_hand_prior_type': 'l2', 'right_hand_prior_type': 'l2', 
        'jaw_prior_type': 'l2', 'use_vposer': True, 'vposer_ckpt': '/home/frg08/Code/smplify-x/V02_05/', 
        'init_joints_idxs': [9, 12, 2, 5], 'body_tri_idxs': [(5, 12), (2, 9)], 'prior_folder': 'priors', 
        'focal_length': 5000.0, 'rho': 100.0, 'interpenetration': False, 'penalize_outside': True, 
        'data_weights': [1.0, 1.0, 1.0, 1.0], 'body_pose_prior_weights': [404.0, 404.0, 57.4, 4.78], 
        'shape_weights': [100.0, 50.0, 10.0, 5.0], 'expr_weights': [100.0, 50.0, 10.0, 5.0], 
        'face_joints_weights': [0.0, 0.0, 0.0, 2.0], 'hand_joints_weights': [0.0, 0.0, 0.0, 2.0], 
        'jaw_pose_prior_weights': None, 'hand_pose_prior_weights': [100.0, 50.0, 10.0, 5.0], 
        'coll_loss_weights': [0.0, 0.0, 0.01, 1.0], 'depth_loss_weight': 100.0, 'df_cone_height': 0.0001, 
        'max_collisions': 128, 'point2plane': False, 'part_segm_fn': '', 
        'ign_part_pairs': ['9,16', '9,17', '6,16', '6,17', '1,2', '12,22'], 'use_hands': False, 'use_face': False, 
        'use_face_contour': False, 'side_view_thsh': 25, 'optim_type': 'lbfgsls', 'lr': 1.0, 'gtol': 1e-09, 
        'ftol': 1e-09, 'maxiters': 30}

output_folder = args.pop('output_folder')
output_folder = osp.expandvars(output_folder)
if not osp.exists(output_folder):
    os.makedirs(output_folder)

# Store the arguments for the current experiment
conf_fn = osp.join(output_folder, 'conf.yaml')
with open(conf_fn, 'w') as conf_file:
    yaml.dump(args, conf_file)

result_folder = args.pop('result_folder', 'results')
result_folder = osp.join(output_folder, result_folder)
if not osp.exists(result_folder):
    os.makedirs(result_folder)

mesh_folder = args.pop('mesh_folder', 'meshes')
mesh_folder = osp.join(output_folder, mesh_folder)
if not osp.exists(mesh_folder):
    os.makedirs(mesh_folder)

out_img_folder = osp.join(output_folder, 'images')
if not osp.exists(out_img_folder):
    os.makedirs(out_img_folder)

float_dtype = args['float_dtype']
if float_dtype == 'float64':
    dtype = torch.float64
elif float_dtype == 'float32':
    dtype = torch.float64
else:
    print('Unknown float type {}, exiting!'.format(float_dtype))
    sys.exit(-1)

use_cuda = args.get('use_cuda', True)
if use_cuda and not torch.cuda.is_available():
    print('CUDA is not available, exiting!')
    sys.exit(-1)

img_folder = args.pop('img_folder', 'images')
dataset_obj = create_dataset(img_folder=img_folder, **args)

start = time.time()

input_gender = args.pop('gender', 'neutral')
gender_lbl_type = args.pop('gender_lbl_type', 'none')
max_persons = args.pop('max_persons', -1)

float_dtype = args.get('float_dtype', 'float32')
if float_dtype == 'float64':
    dtype = torch.float64
elif float_dtype == 'float32':
    dtype = torch.float32
else:
    raise ValueError('Unknown float type {}, exiting!'.format(float_dtype))

joint_mapper = JointMapper(dataset_obj.get_model2data())

FileNotFoundError: [Errno 2] No such file or directory: 'data/images'

In [7]:
focal_length = args.get('focal_length')
camera = create_camera(focal_length_x=focal_length,
                       focal_length_y=focal_length,
                       dtype=dtype,
                       **args)

In [9]:
print(camera)

PerspectiveCamera()


In [10]:
torch.zeros([3, 2], dtype=dtype)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

In [11]:
torch.eye(3, dtype=dtype).unsqueeze(dim=0).repeat(4, 1, 1)

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], dtype=torch.float64)

In [14]:
torch.eye(3, dtype=dtype)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=torch.float64)

In [19]:
torch.index_select(torch.Tensor([10,20,30]), 0, torch.tensor([1,0,2]))

tensor([20., 10., 30.])

In [22]:
[1, ] * 5

[1, 1, 1, 1, 1]

In [ ]:
joints_2d, _3d 
 tensor([[[ 553.9140,   91.7751],
         [ 553.6980,  206.5970],
         [ 453.6500,  200.5350],
         [ 386.1690,  347.8400],
         [ 444.9110,  456.5590],
         [ 653.9280,  221.2060],
         [ 739.2170,  338.8270],
         [ 692.1270,  456.5860],
         [ 512.6100,  494.9150],
         [ 450.8240,  480.2410],
         [ 306.6270,  633.2980],
         [ 244.8870,  859.9170],
         [ 577.3600,  506.7720],
         [ 577.3970,  753.9320],
         [ 580.3100,  951.1510],
         [ 533.3210,   77.0282],
         [ 577.3280,   79.7950],
         [ 506.6860,   85.9202],
         [ 600.9330,   94.6972],
         [ 595.0250, 1001.1000],
         [ 615.6630,  995.2380],
         [ 574.3210,  960.0350],
         [ 194.6770,  901.1330],
         [ 185.9320,  883.5470],
         [ 250.6830,  880.5590],
         [ 553.9140,   91.7751],
         [ 553.6980,  206.5970],
         [ 453.6500,  200.5350],
         [ 386.1690,  347.8400],
         [ 444.9110,  456.5590],
         [ 653.9280,  221.2060],
         [ 739.2170,  338.8270],
         [ 692.1270,  456.5860]]]) 
 tensor([[[-4.5038e-01,  1.0202e-04,  2.5394e-01],
         [-4.5038e-01,  1.0202e-04,  2.5394e-01],
         [-3.2989e-01,  1.0202e-04,  2.5903e-01],
         [-2.2517e-01,  1.0203e-04,  7.7453e-02],
         [-8.8651e-02,  1.0203e-04,  5.9314e-02],
         [ 3.7028e-02,  1.0203e-04,  5.9569e-02],
         [ 1.7218e-01,  1.0203e-04,  2.2672e-01],
         [ 3.0644e-01,  1.7191e-01,  1.2355e-01],
         [ 3.0661e-01,  1.5211e-01, -7.8142e-02],
         [ 3.0426e-01,  1.0705e-01, -3.1296e-01],
         [ 3.5514e-01,  1.0823e-01, -4.3810e-01],
         [ 3.0644e-01, -1.7171e-01,  1.2355e-01],
         [ 3.0661e-01, -1.5191e-01, -7.8142e-02],
         [ 3.0426e-01, -1.0685e-01, -3.1296e-01],
         [ 3.5514e-01, -1.0802e-01, -4.3810e-01],
         [ 4.0271e-01,  1.0202e-04,  2.9802e-01],
         [ 6.0081e-01,  1.0203e-04,  2.9027e-01],
         [-5.1785e-01,  9.8666e-02,  1.9752e-01],
         [-4.3026e-01,  1.2439e-01, -4.6530e-02],
         [-4.9236e-01,  1.0782e-01, -3.1861e-01],
         [-4.5265e-01,  1.1573e-01, -5.1001e-01],
         [-5.1785e-01, -9.8462e-02,  1.9752e-01],
         [-4.3026e-01, -1.2418e-01, -4.6530e-02],
         [-4.9236e-01, -1.0761e-01, -3.1861e-01],
         [-4.5265e-01, -1.1553e-01, -5.1001e-01],
         [-5.7836e-01,  1.0203e-04,  1.8290e-01],
         [-7.2306e-01,  1.0203e-04,  1.8111e-01],
         [-7.9729e-01,  1.0203e-04,  1.8730e-01],
         [-9.0279e-01,  1.0203e-04,  1.9364e-01],
         [-9.9845e-01,  1.0203e-04,  1.9758e-01],
         [-1.0928e+00,  1.0203e-04,  2.1122e-01],
         [-1.1275e+00,  1.0203e-04,  2.1015e-01],
         [ 7.3855e-01,  1.0202e-04,  1.9284e-01]]]) 


In [33]:
ls = [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]

In [35]:
import numpy as np
ls = np.array(ls)[:,0]

In [37]:
[i for i, x in enumerate(ls) if x]

[10, 25]